# 10_Silver: サインインイベントの変換と強化

## 概要
このノートブックでは、Bronze層のデータをビジネスロジックに基づいて変換し、Silver層のテーブルを作成します。

## 処理フロー
1. Bronze層からデータを読み込み
2. ビジネスに有用な派生カラムを追加
3. セキュリティ分析のための特徴量を計算
4. Impossible Travel（物理的に不可能な移動）を検出

## メダリオンアーキテクチャ - Silver層
- **目的**: データをビジネス価値のある形式に変換し、分析可能な状態にする
- **特徴**: クリーニング、正規化、エンリッチメント、ビジネスルールの適用
- **利点**: 分析チームが直接利用できる高品質なデータを提供

## セキュリティ分析の観点
- **サインイン成功/失敗の判定**: 異常な失敗率を検出
- **MFA（多要素認証）の使用**: セキュリティレベルの評価
- **時間帯分析**: 営業時間外のアクセスを特定
- **地理的異常検出**: 短時間での異なる国/都市からのアクセスを検出


## Step 1: Silverテーブルの作成

Bronze層のデータから、以下の処理を段階的に実施してSilverテーブルを作成します：

### 処理の段階

**1. base CTE（共通テーブル式）**
- タイムスタンプの解析と時間帯の抽出
- 型変換（文字列→真偽値、整数）
- ビジネスフラグの作成：
  - `is_success` / `is_failure`: サインインの成功/失敗
  - `is_mfa`: MFAが使用されたか
  - `ca_failed`: 条件付きアクセスの失敗
  - `is_off_hours`: 営業時間外（20時～翌8時）のアクセス

**2. travel CTE**
- ウィンドウ関数（`LAG`）を使用して、同一ユーザーの前回のサインイン情報を取得
- 前回のサインイン国/都市とタイムスタンプを保持

**3. 最終SELECT**
- **Impossible Travel** の判定：
  - 前回と異なる国または都市からのサインイン
  - かつ、前回のサインインから1時間以内（3600秒）
  - この条件を満たす場合、物理的に不可能な移動と判定


In [0]:
USE CATALOG users;
USE users.yukiteru_koide;

CREATE OR REPLACE TABLE users.yukiteru_koide.signin_events_silver AS
WITH base AS (
  SELECT
    id,
    to_timestamp(createdDateTime) AS created_ts,
    to_date(to_timestamp(createdDateTime)) AS event_date,
    hour(to_timestamp(createdDateTime)) AS event_hour,
    userPrincipalName,
    userId,
    appDisplayName,
    appId,
    ipAddress,
    clientAppUsed,
    CASE lower(isInteractive) WHEN 'true' THEN true WHEN 'false' THEN false ELSE NULL END AS isInteractive,
    authenticationRequirement,
    conditionalAccessStatus,
    deviceOperatingSystem,
    browser,
    locationCountryOrRegion,
    locationCity,
    CAST(statusErrorCode AS INT) AS statusErrorCode,
    statusFailureReason,
    statusAdditionalDetails,
    (statusErrorCode = 0) AS is_success,
    (statusErrorCode <> 0) AS is_failure,
    (authenticationRequirement = 'multiFactorAuthentication') AS is_mfa,
    (conditionalAccessStatus = 'failure') AS ca_failed,
    (hour(to_timestamp(createdDateTime)) < 8 OR hour(to_timestamp(createdDateTime)) >= 20) AS is_off_hours
  FROM users.yukiteru_koide.signin_logs_bronze
),
travel AS (
  SELECT
    *,
    LAG(locationCountryOrRegion) OVER (PARTITION BY userId ORDER BY created_ts) AS prev_country,
    LAG(locationCity) OVER (PARTITION BY userId ORDER BY created_ts) AS prev_city,
    LAG(created_ts) OVER (PARTITION BY userId ORDER BY created_ts) AS prev_ts
  FROM base
)
SELECT
  *,
  CASE
    WHEN prev_country IS NOT NULL
     AND (locationCountryOrRegion <> prev_country OR locationCity <> prev_city)
     AND (unix_timestamp(created_ts) - unix_timestamp(prev_ts)) <= 3600
    THEN true ELSE false
  END AS impossible_travel
FROM travel;


## Step 2: テーブルの最適化

Silverテーブルを最適化して、クエリパフォーマンスを向上させます。


In [0]:
OPTIMIZE users.yukiteru_koide.signin_events_silver;
